In [1]:
import os, os.path, re
import numpy as np
from sklearn.preprocessing import normalize, scale, MinMaxScaler
from collections import Counter
from sys import getsizeof
import time
from sklearn.cluster import *

** Data preprocessing **

Extract filenames and reorder it numerically.

In [57]:
files_path = 'collected_models2/'
ts_labels = ['chest_volume', 'heart_rate', 'oxygen_concentration']
ts_labels = sorted(ts_labels)

number_ts_pieces = len(os.listdir(files_path)) / len(ts_labels)
# if the number is not divided 
if abs(number_ts_pieces - round(number_ts_pieces)) > 0:
    print('ERROR: invalid number of files in ', files_path)
else:
    number_ts_pieces = int(number_ts_pieces)    

def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):    
    return [ atoi(c) for c in re.split('(\d+)', text) ]

file_names = sorted(os.listdir(files_path), key=natural_keys)

** Extract models from files **

After the extraction we preprocess them. Eliminate parameters and refactor the variable

In [65]:
def get_models_from_file(number_of_file, type_of_ts):
    index_of_ts = ts_labels.index(type_of_ts)
    file_name = file_names[number_ts_pieces * index_of_ts + number_of_file]        
    file_opened = open(files_path + file_name, 'r')
    
    func_and_params_names = file_opened.readlines()
    models = []
    for entity in func_and_params_names:          
        entity = entity.split(' ')[-1]
        models.append(entity)
        models[-1] = models[-1].strip()                    
        models[-1] = re.sub(r'X\[(\d+)\]', r'x\1', models[-1])    
    return models

Write functions responsible for conversion of a string-handle to a tree matrix and codings.

In [66]:
def find_number_of_tokens(handle):    
    counter_tokens = 0
    counter_variables = 0

    for i in range(len(handle)):
        if handle[i] == '_': 
            counter_tokens += 1
        elif i < len(handle)-1 and handle[i] == 'x' and handle[i+1].isdigit():
            counter_variables += 1;

    return (counter_tokens, counter_variables)

In [67]:
def create_map_tokens_params():
    file_opened = open('data/numbParam.txt', 'r')
    primitives_lines = file_opened.readlines()
    tokens_codes = {line.split()[0] : int(ind) for ind,line in enumerate(primitives_lines)}    
    tokens_params = {line.split()[0] : int(line.split()[1]) for line in primitives_lines}    
    return (tokens_codes, tokens_params)

In [68]:
def dfs_search_on_handle(handle):
    counters = find_number_of_tokens(handle)
    number_tokens = counters[0] + counters[1]
    
    waiting_tokens = []
    encodings = np.zeros(number_tokens, dtype = int)        
    current_token, left, right = 0, 0, 0
    is_a_token_processed_now = False    
    
    map_tokens_params = create_map_tokens_params()[0]
    
    for right in range(len(handle)):
        if handle[right] == '_':
            # the root is detected
            waiting_tokens.append(current_token)
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            right += 1
            break;  
    
    matr = [[] for i in range(number_tokens)]            
    
    # now process the remaining vertices
    reserved_right = right
    for right in np.arange(right, len(handle)):
        if handle[right] == ')':
            waiting_tokens.pop()        
    
        if not is_a_token_processed_now and handle[right].isalpha():
            is_a_token_processed_now = True
            left = right
    
        # if a token is found
        if handle[right] == '_':
            # new token is detected
            current_token += 1
            matr[waiting_tokens[-1]].append(current_token)
            waiting_tokens.append(current_token)
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            is_a_token_processed_now = False      
        
        # if a variable is found
        if right < len(handle)-1 and handle[right] == 'x' and handle[right+1].isdigit():
            # new variable is detected
            current_token += 1
            matr[waiting_tokens[-1]].append(current_token)
            while right < len(handle)-1 and handle[right] == 'x' and handle[right+1].isdigit():
                right += 1
            token = handle[left:right + 1]
            encodings[current_token] = map_tokens_params[token]
            is_a_token_processed_now = False            
    
    return (matr, encodings)

In [69]:
def incidence_to_adjacency(incidence):
    size_of_mat = len(incidence[0])
    adjacency = np.zeros((size_of_mat, size_of_mat))    
    for ind, row in enumerate(incidence[0]):
        adjacency[ind][row] = 1
    return adjacency

** Collect primitive structural features from a population of models**

In [106]:
def get_simple_features_from_ts(number_of_file, type_of_ts, tokens_codes):
    models = get_models_from_file(number_of_file, type_of_ts)
    
    primitive_frequences = np.zeros(len(tokens_codes) + 1)
    lower_bound_code_variables = tokens_codes['x0']

    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        model_primitive_frequences = Counter(encodings)
        for key in model_primitive_frequences:
            primitive_frequences[key] += model_primitive_frequences[key]
        primitive_frequences[-1] += len(encodings)
    primitive_frequences[-1] = primitive_frequences[-1] / len(models)
    #return normalize(primitive_frequences.reshape(-1,1), axis=0)
    #return primitive_frequences.reshape(-1,1)
    
    return scale(primitive_frequences.reshape(-1,1), axis=0)

In [113]:
tokens_codes, _ = create_map_tokens_params()
feature_matrices_of_ts = {label : np.zeros((len(tokens_codes) + 1, number_ts_pieces)) for label in ts_labels}

for label in ts_labels:
    for index in range(number_ts_pieces):        
        feature_matrices_of_ts[label][:,index] = get_simple_features_from_ts(index, label, tokens_codes)[:,0]
        
#feature_matrices_of_ts = {label : scale(feature_matrices_of_ts[label].T) for label in ts_labels}        
feature_matrices_of_ts = {label : feature_matrices_of_ts[label].T for label in ts_labels}        

In [114]:
unite_feature_matrix = np.vstack((feature_matrices_of_ts[label] for label in ts_labels))

** Now perform two-class classification **

In [115]:
unite_feature_matrix.shape

(1020, 37)

In [153]:
ts_labels = ['chest_volume', 'heart_rate', 'oxygen_concentration']
which_label_is_positive = ts_labels[0]

target_vector = np.zeros((unite_feature_matrix.shape[0],1))
ts_labels_in_order_from_dictionary = [label for label in feature_matrices_of_ts]
index_of_label_positive = ts_labels_in_order_from_dictionary.index(which_label_is_positive)
all_inidices_of_samples = np.arange(target_vector.shape[0])
positive_indices_samples = number_ts_pieces * index_of_label_positive + np.arange(number_ts_pieces)
negative_positive_samples = [ind for ind in all_inidices_of_samples if not ind in positive_indices_samples]
target_vector[number_ts_pieces * index_of_label_positive:number_ts_pieces * (index_of_label_positive + 1)] = np.ones((number_ts_pieces,1))
backup_target = target_vector

Divide set of samples on training and test.

In [154]:
fraction_of_test_samples = 0.3

indices_of_test_sample = np.random.choice([True, False], len(all_inidices_of_samples), p = [fraction_of_test_samples, 1 - fraction_of_test_samples])
target_vector = 2* backup_target - 1 
train_matrix = unite_feature_matrix[~indices_of_test_sample,:]
test_matrix = unite_feature_matrix[indices_of_test_sample,:]
train_target = target_vector[~indices_of_test_sample].reshape(sum(~indices_of_test_sample),)
test_target = target_vector[indices_of_test_sample].reshape(sum(indices_of_test_sample),)

In [155]:
target_vector[number_ts_pieces + 5]

array([-1.])

In [156]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(train_matrix, train_target) 
print(clf.n_support_)
predictions = clf.predict(test_matrix)
#print("predictions: ", predictions)
errors = predictions != test_target
print("error = ", sum(errors) / len(errors))
print("error on positive = ", sum(errors[test_target == max(test_target)]) / len(errors[test_target == max(test_target)]))
print("error on negative = ", sum(errors[test_target == min(test_target)]) / len(errors[test_target == min(test_target)]))

[89 92]
error =  0.0647249190939
error on positive =  0.0
error on negative =  0.0995024875622


In [139]:
from sklearn.svm import SVC
clf = SVC(kernel='linear')
clf.fit(train_matrix, train_target) 
print(clf.n_support_)
predictions = clf.predict(train_matrix)
errors = predictions != train_target
print("error = ", sum(errors) / len(errors))
print("error on positive = ", sum(errors[train_target == max(train_target)]) / len(errors[train_target == max(train_target)]))
print("error on negative = ", sum(errors[train_target == min(train_target)]) / len(errors[train_target == min(train_target)]))

[90 90]
error =  0.0591630591631
error on positive =  0.0
error on negative =  0.0889370932755


In [86]:
from sklearn import neighbors, datasets



n_neighbors = 17
for weights in ['uniform', 'distance']:
    # we create an instance of Neighbours Classifier and fit the data.
    clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
    clf.fit(train_matrix, train_target)

    predictions = clf.predict(np.c_[test_matrix])
    print("weights: ", weights)
    errors = predictions != test_target
    print("error = ", sum(errors) / len(errors))
    print("error on positive = ", sum(errors[test_target == max(test_target)]) / len(errors[test_target == max(test_target)]))
    print("error on negative = ", sum(errors[test_target == min(test_target)]) / len(errors[test_target == min(test_target)]))


weights:  uniform
error =  0.16
error on positive =  0.29702970297
error on negative =  0.0804597701149
weights:  distance
error =  0.167272727273
error on positive =  0.316831683168
error on negative =  0.0804597701149


In [109]:
from sklearn import neighbors, datasets



h = .02  # step size in the mesh
error = np.inf
where_min = -1

for n_neighbors in 1 + np.arange(30):
    for weights in ['uniform', 'distance']:
        # we create an instance of Neighbours Classifier and fit the data.
        clf = neighbors.KNeighborsClassifier(n_neighbors, weights=weights)
        clf.fit(train_matrix, train_target)

        predictions = clf.predict(np.c_[test_matrix])
        errors = predictions != test_target
        if sum(errors) / len(errors) < error:
            where_min = n_neighbors
            error = sum(errors) / len(errors)
            

where_min, error

(17, 0.2893772893772894)

In [91]:
from sklearn import preprocessing
B = np.array([[5,-5],[0,2]])
print(np.mean(B,axis=0))
B = B-np.mean(B,axis=0)
print(B)
print(np.max(B,axis=0))
print(np.divide(B,np.max(B,axis=0)))
preprocessing.scale(B,axis = 0)

[ 2.5 -1.5]
[[ 2.5 -3.5]
 [-2.5  3.5]]
[ 2.5  3.5]
[[ 1. -1.]
 [-1.  1.]]


array([[ 1., -1.],
       [-1.,  1.]])

** Collect DFS-code features **

In [87]:
def levenstein_string_distance(source, target):
    if len(source) < len(target):
        return levenshtein(target, source)

    # So now we have len(source) >= len(target).
    if len(target) == 0:
        return len(source)

    # We call tuple() to force strings to be used as sequences
    # ('c', 'a', 't', 's') - numpy uses them as values by default.
    source = np.array(tuple(source))
    target = np.array(tuple(target))

    # We use a dynamic programming algorithm, but with the
    # added optimization that we only need the last two rows
    # of the matrix.
    previous_row = np.arange(target.size + 1)
    for s in source:
        # Insertion (target grows longer than source):
        current_row = previous_row + 1

        # Substitution or matching:
        # Target and source items are aligned, and either
        # are different (cost of 1), or are the same (cost of 0).
        current_row[1:] = np.minimum(
                current_row[1:],
                np.add(previous_row[:-1], target != s))

        # Deletion (target grows shorter than source):
        current_row[1:] = np.minimum(
                current_row[1:],
                current_row[0:-1] + 1)

        previous_row = current_row

    return previous_row[-1]

In [88]:
def distance_between_populations(population_first, population_second):
    len_first, len_second = len(population_first), len(population_second)
    # calculate sum of distances for all pairs of models: one from first population, the other from second
    cumulative_distance = 0
    for model_from_first in population_first:
        for model_from_second in population_second:
            cumulative_distance = cumulative_distance + levenstein_string_distance(model_from_first, model_from_second)
    cumulative_distance = cumulative_distance / (len_first * len_second)
    return cumulative_distance

In [89]:
def get_simple_features_from_ts(number_of_file, type_of_ts, tokens_codes):
    models = get_models_from_file(number_of_file, type_of_ts)
    
    primitive_frequences = np.zeros(len(tokens_codes))    

    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        primitive_frequences = encodings

    return primitive_frequences.reshape(-1,1)
    #return primitive_frequences.reshape(-1,1)

In [90]:
def create_matrix_of_codes_of_one_population(number_of_file, type_of_ts, tokens_codes):
    codes = 'QWERTYUIOPASDFGHJKLZXCVBNM123456789/*-+=?!'
    models = get_models_from_file(number_of_file, type_of_ts)
    
    matrix_representation = []    
    for model in models:
        matr, encodings = dfs_search_on_handle(model)
        encodings = np.array(encodings)
        if len(model) == 0:
            break
        matrix_representation.append(''.join(np.array(list(codes))[encodings]))

    return matrix_representation
    #return primitive_frequences.reshape(-1,1)

In [91]:
tokens_codes, _ = create_map_tokens_params()
feature_matrices_of_ts = {label : [] for label in ts_labels}
    
for label in ts_labels:
    for index in range(number_ts_pieces):
        feature_matrices_of_ts[label].append(create_matrix_of_codes_of_one_population(index, label, tokens_codes))

In [92]:
distances_between_segments = np.zeros((len(ts_labels) * number_ts_pieces, len(ts_labels) * number_ts_pieces))


start = time.time()
for ind_label_f,label_f in enumerate(feature_matrices_of_ts):
    for ind_label_s,label_s in enumerate(feature_matrices_of_ts):
        print(label_f, label_s)
        for ind_population_f,population_f in enumerate(feature_matrices_of_ts[label_f]):
            for ind_population_s,population_s in enumerate(feature_matrices_of_ts[label_s]):
                index_first = ind_label_f * number_ts_pieces + ind_population_f
                index_second = ind_label_s * number_ts_pieces + ind_population_s
                distances_between_segments[index_first][index_second] = distance_between_populations(population_f[0], population_s[0])
    
end = time.time()
print(end - start)

chest_volume chest_volume


KeyboardInterrupt: 

In [43]:
random_state = 50
clusters = KMeans(n_clusters=3, random_state=random_state,n_init = 10).fit_predict(distances_between_segments)
true_clusters = np.zeros(len(ts_labels) * number_ts_pieces)
for ind,_ in enumerate(ts_labels):
    true_clusters[ind * number_ts_pieces:(ind + 1) * number_ts_pieces] = ind * np.ones(number_ts_pieces)
print(sum(clusters != true_clusters) / len(clusters))
clusters

0.716666666667


array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [26]:
random_state = 170
clusters = AgglomerativeClustering(n_clusters=3).fit_predict(distances_between_segments)
print(sum(clusters != true_clusters) / len(clusters))
clusters

0.791176470588


array([0, 2, 2, ..., 0, 0, 0])

In [79]:
l = [0,2,3,4]
l[1:2]

[2]

In [ ]:
fraction_of_test_samples = 0.3

test_indices = np.random.choice([True, False], number_ts_pieces, p = [fraction_of_test_samples, 1 - fraction_of_test_samples])
target_vector = 2* target_vector - 1 

train_matrix = unite_feature_matrix[~indices_of_test_sample,:]
test_matrix = unite_feature_matrix[indices_of_test_sample,:]
train_target = target_vector[~indices_of_test_sample].reshape(sum(~indices_of_test_sample),)
test_target = target_vector[indices_of_test_sample].reshape(sum(indices_of_test_sample),)


3